In [1]:
import pandas as pd
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from PIL import Image
import matplotlib.pyplot as plt
import os
from IPython.display import display

# Cargar el dataset
df = pd.read_csv('combined_output_radical.csv')  

example_row = df.iloc[0]
file_path = "imagenes/resizeSD/" + example_row['file_name']
prompt = example_row['sdxl_prompt']

print(f"Procesando: {file_path}")
print(f"Prompt: {prompt}")

# Cargar el modelo
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

# Mover a GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)

# Función para procesar una imagen
def process_image(image_path, prompt, strength=0.75, guidance_scale=7.5, num_inference_steps=50):
    """
    Procesa una imagen con SDXL usando image-to-image.
    
    Parámetros:
    - image_path: Ruta a la imagen de entrada
    - prompt: Texto que guía la generación
    - strength: Qué tanto se modificará la imagen (0-1)
    - guidance_scale: Cuánto seguir el prompt (valores típicos: 7-9)
    - num_inference_steps: Número de pasos de inferencia
    
    Devuelve la imagen generada
    """
    try:
        # Cargar y preparar la imagen
        init_image = Image.open(image_path).convert("RGB")
        
        # Mostrar imagen original
        plt.figure(figsize=(10, 10))
        plt.imshow(init_image)
        plt.title("Imagen Original")
        plt.axis('off')
        plt.show()
        
        # Generar la nueva imagen
        image = pipe(
            prompt=prompt,
            image=init_image,
            strength=strength,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
        ).images[0]
        
        # Mostrar la imagen generada
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.title(f"Imagen Generada\nStrength: {strength}, Guidance: {guidance_scale}, Steps: {num_inference_steps}")
        plt.axis('off')
        plt.show()
        
        return image
    
    except Exception as e:
        print(f"Error al procesar la imagen: {e}")
        return None

# Probar con diferentes parámetros para ver el efecto
strengths = [0.6, 0.75, 0.9]  # Cuánto modificar la imagen original
guidance_scales = [7.5, 9.0]  # Qué tanto seguir el prompt

for strength in strengths:
    for guidance in guidance_scales:
        print(f"\nPrueba con strength={strength}, guidance_scale={guidance}")
        result = process_image(
            file_path, 
            prompt, 
            strength=strength,
            guidance_scale=guidance,
            num_inference_steps=50
        )

# Función para procesar varias imágenes del dataset
def process_dataset_sample(df, num_samples=3, strength=0.75, guidance_scale=8.5, steps=50):
    """
    Procesa varias imágenes del dataset
    """
    # Tomar algunas muestras aleatorias
    sample_df = df.sample(num_samples) if len(df) > num_samples else df
    
    for idx, row in sample_df.iterrows():
        print(f"\nProcesando imagen {idx+1}/{len(sample_df)}")
        file_path = row['file_name']
        prompt = row['sdxl_prompt']
        
        print(f"Archivo: {file_path}")
        print(f"Prompt: {prompt}")
        
        result = process_image(
            file_path,
            prompt,
            strength=strength,
            guidance_scale=guidance_scale,
            num_inference_steps=steps
        )

recommended_params = {
    'strength': 0.85,         # Alto para permitir cambios significativos
    'guidance_scale': 9.0,    # Fuerte adherencia al prompt
    'num_steps': 60           # Más pasos para mejor calidad
}

print("\n--- Prueba con parámetros recomendados ---")
print(f"Strength: {recommended_params['strength']}")
print(f"Guidance Scale: {recommended_params['guidance_scale']}")
print(f"Steps: {recommended_params['num_steps']}")

result = process_image(
    file_path,
    prompt,
    strength=recommended_params['strength'],
    guidance_scale=recommended_params['guidance_scale'],
    num_inference_steps=recommended_params['num_steps']
)

Procesando: imagenes/resizeSD/ Impressionism/william-merritt-chase_still-life-with-cockatoo_resize1024.jpg
Prompt: Revolutionize a still life with a white bird on a vase. Transmute it into a cyberpunk dystopia. Overhaul the composition into a sprawling holographic cityscape. Transform the color palette to neon blues and electric pinks. Reimagine the subject in a futuristic lab, with advanced technology and sleek design. A profound transformation where nature meets artificial intelligence in a neon-lit, high-tech environment.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]


Prueba con strength=0.6, guidance_scale=7.5
Error al procesar la imagen: [Errno 2] No such file or directory: 'imagenes/resizeSD/ Impressionism/william-merritt-chase_still-life-with-cockatoo_resize1024.jpg'

Prueba con strength=0.6, guidance_scale=9.0
Error al procesar la imagen: [Errno 2] No such file or directory: 'imagenes/resizeSD/ Impressionism/william-merritt-chase_still-life-with-cockatoo_resize1024.jpg'

Prueba con strength=0.75, guidance_scale=7.5
Error al procesar la imagen: [Errno 2] No such file or directory: 'imagenes/resizeSD/ Impressionism/william-merritt-chase_still-life-with-cockatoo_resize1024.jpg'

Prueba con strength=0.75, guidance_scale=9.0
Error al procesar la imagen: [Errno 2] No such file or directory: 'imagenes/resizeSD/ Impressionism/william-merritt-chase_still-life-with-cockatoo_resize1024.jpg'

Prueba con strength=0.9, guidance_scale=7.5
Error al procesar la imagen: [Errno 2] No such file or directory: 'imagenes/resizeSD/ Impressionism/william-merritt-chase

In [ ]:
import torch

import gc

gc.collect()
torch.cuda.empty_cache()

: 